In [2]:
# set python to pick up local directory
from pathlib import Path
import sys
import os
path_root = str(Path(os.getcwd()).parents[1])
sys.path.append(path_root)
# sys.path.append(package_root)

# auto reload
%reload_ext autoreload
%autoreload 2

# hush
import warnings
warnings.simplefilter('always', category=FutureWarning)

In [3]:
import pandas as pd
import numpy as np
import json
import requests
from FPLmanager.data.fplapi import *
import time

## Read in Historical Data

In [187]:
team_list = pd.read_csv('clean/teams.csv')

### 2016, 2017

Replace missing team values

In [258]:
season_ids = pd.read_csv("raw/season_ids.csv")

In [247]:
games_1622 = pd.read_csv("raw/cleaned_merged_seasons.csv")
games_1622 = games_1622.rename(columns={"season_x": "season", "team_x": "team"})
games_1618 = games_1622[games_1622['season'].isin(["2016-17", "2017-18"])]
games_2022 = games_1622[games_1622['season'].isin(["2020-21", "2021-22"])]

games_1618['date'] = pd.to_datetime(games_1618['kickoff_time'])
games_1618 = games_1618.sort_values(by=['date'])
games_1618['date'] = games_1618['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

games_1618['home_team'] = games_1618.apply(lambda x: x['team'] if x['was_home'] else x['opp_team_name'], axis=1)
games_1618['away_team'] = games_1618.apply(lambda x: x['team'] if not x['was_home'] else x['opp_team_name'], axis=1)

/var/folders/3d/tnk1dqns7kd6x1yqw6m752z00000gn/T/ipykernel_31069/1253842148.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  games_1622 = pd.read_csv("raw/cleaned_merged_seasons.csv")
/var/folders/3d/tnk1dqns7kd6x1yqw6m752z00000gn/T/ipykernel_31069/1253842148.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_1618['date'] = pd.to_datetime(games_1618['kickoff_time'])


In [248]:
fixtures_1617 = pd.read_csv("raw/fixtures_1617.csv")[['date', 'home_team', 'away_team']]

missing_merged = games_1618.merge(fixtures_1617, on=['date'], how='left')
missing_merged = missing_merged[
    (missing_merged['home_team_x'] == missing_merged['home_team_y']) | 
    (missing_merged['away_team_x'] == missing_merged['away_team_y'])
].reset_index(drop=True)

missing_merged['team'] = missing_merged.apply(
    lambda x: x['home_team_y'] if x['was_home'] else x['away_team_y'], axis=1
)
games_1618_corrected = missing_merged.drop(columns=['date', 'home_team_x', 'away_team_x', 'home_team_y', 'away_team_y'])

### Covid: 2019 - 2020

In [249]:
teams_1920 = team_list[team_list.season == "2019-20"][['team', 'team_name']]
fixtures_1920 = pd.read_csv("raw/fixtures_1920.csv")

fixtures_1920 = fixtures_1920.merge(teams_1920, left_on='team_h', right_on='team', how='left')
fixtures_1920 = fixtures_1920.merge(teams_1920, left_on='team_a', right_on='team', how='left')
fixtures_1920 = fixtures_1920.rename(columns={"id": "fixture", "team_name_x": "home_team", "team_name_y": "away_team"})
fixtures_1920 = fixtures_1920[['fixture', 'home_team', 'away_team']]

In [250]:
# fix weird gameweeks
games_1920 = pd.read_csv("raw/gw_1920.csv")
games_1920['round'] = games_1920['round'].apply(lambda x: x if x < 30 else x-9)
games_1920['gw'] = games_1920['gw'].apply(lambda x: x if x < 30 else x-9)

# fix player names
games_1920['name'] = games_1920['name'].apply(lambda x: " ".join(x.split('_')[:-1]))

# merge with fixtures to get team names
games_1920 = games_1920.merge(fixtures_1920, on='fixture', how='left')
games_1920['team'] = games_1920.apply(lambda x: x['home_team'] if x['was_home'] else x['away_team'], axis=1)
games_1920['opp_team_name'] = games_1920.apply(lambda x: x['away_team'] if x['was_home'] else x['home_team'], axis=1)
games_1920 = games_1920.drop(columns=['home_team', 'away_team'])
games_1920['season'] = "2019-20"

In [257]:
games_1920.columns

Index(['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'team', 'opp_team_name', 'season'],
      dtype='object')

### 2023 - 2024

In [251]:
teams_2324 = team_list[team_list.season == "2023-24"][['team', 'team_name']]
fixtures_2324 = pd.read_csv("raw/fixtures_2324.csv")

fixtures_2324 = fixtures_2324.merge(teams_2324, left_on='team_h', right_on='team', how='left')
fixtures_2324 = fixtures_2324.merge(teams_2324, left_on='team_a', right_on='team', how='left')
fixtures_2324 = fixtures_2324.rename(columns={"id": "fixture", "team_name_x": "home_team", "team_name_y": "away_team"})
fixtures_2324 = fixtures_2324[['fixture', 'home_team', 'away_team']]

In [252]:
games_2324 = pd.read_csv("raw/gw_2324.csv")
games_2324 = games_2324.rename(columns={"season_x": "season", "team_x": "team"})
games_2324 = games_2324.drop(columns=[
    'expected_assists', 'expected_goals', 'expected_goal_involvements', 'expected_goals_conceded', 'starts', 'xP'
])

games_2324 = games_2324.merge(fixtures_2324, on='fixture', how='left')
games_2324['team'] = games_2324.apply(lambda x: x['home_team'] if x['was_home'] else x['away_team'], axis=1)
games_2324['opp_team_name'] = games_2324.apply(lambda x: x['away_team'] if x['was_home'] else x['home_team'], axis=1)
games_2324 = games_2324.drop(columns=['home_team', 'away_team'])
games_2324['season'] = "2023-24"

In [246]:
set(sorted(games_2324.columns)) - set(sorted(games_1920.columns))

{'position'}

### Merge Across Seasons and Apply Unique Codes

In [266]:
season_ids = pd.read_csv("raw/season_ids.csv")
season_ids = season_ids[["first_name", "second_name", "id", "code", "season"]]
season_ids['name'] = season_ids["first_name"] + " " + season_ids["second_name"]
season_ids = season_ids.drop(columns=["first_name", "second_name"])
season_ids = season_ids.rename(columns={"id": "element"})

In [314]:
# games = pd.concat([games_1618_corrected, games_1920, games_2022, games_2324], ignore_index=True)
player_history = pd.concat([games_1618_corrected, games_2022, games_2324], ignore_index=True)
player_history['position'] = player_history.position.apply(lambda x: 'GK' if x == 'GKP' else x)
player_history = player_history.merge(season_ids.drop(columns="name"), on=["season", "element"], how="left")
player_history = player_history.sort_values(by=['season', 'gw', 'name']).reset_index(drop=True)
player_history.to_csv("clean/players.csv", index=False)

In [6]:
# pos_mapping = {
#     1: "GK",
#     2: "DEF",
#     3: "MID",
#     4: "FWD"
# }

# players_24 = pd.DataFrame(bootstrap["elements"])[
#     ['first_name', 'second_name', 'code', 'id', 'element_type'] # 'team', 'team_code',
# ].drop_duplicates().reset_index(drop=True)
# players_24["position"] = players_24["element_type"].map(pos_mapping)
# players_24["name"] = players_24["first_name"] + " " + players_24["second_name"]
# players_24 = players_24.rename(columns={"id": "element"})
# players_24 = players_24.drop(columns=["first_name", "second_name", "element_type"])

# player_histories = []
# for player in players_24.iterrows():
#     player = player[1]
#     print(player['name'])
#     player_history = get_player_history(player['element'])['history']
#     player_histories.extend(player_history)

# player_histories = pd.DataFrame(player_histories)

## Create Derived Tables

In [5]:
players = pd.read_csv("clean/players.csv")

### Fixture Information

In [316]:
fixtures = players.copy()[
    ['fixture', 'gw', 'season', 'kickoff_time', 'team', 'opp_team_name', 'was_home']
].drop_duplicates().sort_values(by=['season', 'gw', 'team'])
fixtures['home_team'] = fixtures.apply(lambda x: x['team'] if x['was_home'] else x['opp_team_name'], axis=1)
fixtures['away_team'] = fixtures.apply(lambda x: x['team'] if not x['was_home'] else x['opp_team_name'], axis=1)
fixtures = fixtures.drop(columns=['team', 'opp_team_name', 'was_home'])
fixtures = fixtures.sort_values(by=['season', 'gw', 'fixture']).drop_duplicates().reset_index(drop=True)
fixtures.to_csv("clean/fixtures.csv", index=False)

### Price Information

In [317]:
# subset only to times when player value changes
player_value = players.copy()[['name', 'value', 'element', 'season', 'code', 'gw']].sort_values(
    by=['season', 'gw', 'name']
).reset_index(drop=True)
player_value['value_delta'] = player_value.groupby(['code', 'season'])['value'].diff()
price_changes = player_value[
    (player_value['value_delta'].isna()) | (player_value['value_delta'] != 0)
].reset_index(drop=True)
price_changes = price_changes.drop(columns=['value_delta'])
price_changes.to_csv("clean/prices.csv", index=False)

### Player Information

In [333]:
player_info = players.copy()[['code', 'element', 'position', 'season', 'team', 'gw']]
player_info = player_info.loc[player_info.groupby(['code', 'element', 'position', 'season', 'team'])['gw'].idxmin()]

# just choose a name if they have multiple
player_names = players.copy()[['name', 'code']].sort_values('name')
player_names = player_names.groupby('code')['name'].first().reset_index()

player_info = player_info.merge(player_names, on='code', how='left')
player_info = player_info[['name', 'code', 'element', 'position', 'season', 'team', 'gw']]
player_info = player_info.sort_values(by=['name', 'season', 'gw']).reset_index(drop=True)
player_info.to_csv("clean/player_info.csv", index=False)

## Write to SQL

In [14]:
from FPLmanager.data.db import *
from FPLmanager import DB_URL
from sqlalchemy import create_engine

In [15]:
players = pd.read_csv("clean/players.csv")
fixtures = pd.read_csv("clean/fixtures.csv")
player_info = pd.read_csv("clean/player_info.csv")
prices = pd.read_csv("clean/prices.csv")
teams = pd.read_csv("clean/teams.csv")

In [6]:
# # fixtures update
# max_val = 44470
# min_val = 44470 - 1900
# fixtures['unique_id'] = range(min_val, max_val)
# fixtures['code'] = fixtures.apply(lambda x: int(x['season'][2:4] + str(x['unique_id'])), axis=1)
# fixtures = fixtures.drop(columns=['unique_id'])
# fixtures.to_csv("clean/fixtures.csv", index=False)

In [17]:
db = create_engine(DB_URL) 
conn = db.connect()

players.to_sql('player_gw', conn, if_exists='append', index=False)
fixtures.to_sql('fixtures', conn, if_exists='append', index=False)
player_info.to_sql('players', conn, if_exists='append', index=False)
prices.to_sql('prices', conn, if_exists='append', index=False)
teams.to_sql('teams', conn, if_exists='append', index=False)
conn.close()